In [3]:
import pickle as pkl
from pprint import pprint

class infographicInfo:
    def __init__(self,currow):
        self.views = int(currow['views']);
        self.likes = int(currow['likes']);
        self.shares = int(currow['shares']);
        self.comments = int(currow['comments']);
        self.staffPick = currow['staffPick'];
        self.imgurl = currow['url_img_ful_res'];
        self.mainurl = currow['url'];
        self.title = currow['igTitle'];
        self.category = currow['category'];
        self.description = currow['description'];
        self.title = currow['title'];
        self.tags = currow['tags'];

In [4]:
infographicList = pkl.load(open("infographics60K_metadata.pckl", "rb"))

In [5]:
pprint(infographicList[0].__dict__)

{'category': 'Business',
 'comments': 7,
 'description': "Visually's latest report dives into the relationship between "
                'marketers and creatives.',
 'imgurl': 'http://thumbnails.visually.netdna-cdn.com/marketers-vs-creatives-communication-breakdown-thumbnail_5639568df0acc_h1000.png',
 'imname': 'marketers-vs-creatives-communication-breakdown-thumbnail_5639568df0acc.png',
 'likes': 150,
 'mainurl': 'http://visual.ly/marketers-vs-creatives-communication-breakdown',
 'shares': 75,
 'staffPick': 'FALSE',
 'tags': ['marketing', 'content', 'markting in business', 'designers'],
 'title': 'Marketers vs Creatives: Communication Breakdown',
 'transcript': [],
 'views': 8760}
